In [12]:
using DynamicPolynomials, SparseArrays
using LinearAlgebra


println("***Problem setting***")

m=5
r=m

function getmat(m,r)

    A=2*rand(Float64,r,r).-1
    B=2*rand(Float64,r,r).-1
    C=2*rand(Float64,r,r).-1
    D=2*rand(Float64,r,r).-1

    M=zeros(Float64,m*r,m*r)

    for i in 1:m, j in 1:i
        if i==j
            M[r*(i-1)+1:i*r,r*(j-1)+1:j*r]=D
        else
            M[r*(i-1)+1:i*r,r*(j-1)+1:j*r]=C*A^(i-j-1)*B
        end
    end

    return m*r, Symmetric(M'*M)
end

#n,A=getmat(m,r)
n=m^2; include("./mat_size$(n).jl")

println("Number of variable: n=",n)
println("====================")

@polyvar x[1:n]# variables
f=sum(-A[i,j]*x[i]*x[j] for i=1:n for j=1:n)

g=Vector{Polynomial{true,Float64}}([])
 #type of coefficients of each polynomial must be float

m=length(g)
println("Number of inequality constraints: m=",m)
println("====================")

h=[1.0-sum(x.^2)]

l=length(h)
println("Number of equality constraints: l=",l)

***Problem setting***
Number of variable: n=25
Number of inequality constraints: m=0
Number of equality constraints: l=1


In [14]:
include("../src/InterruptedRelax.jl")
using .InterruptedRelax

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=InterruptedRelax.get_info(x,f,g,h,sparse=false);


k=1

println("Relaxation order: k=",k)

s=50

println("Block size: s=",s)

@time opt_val1,opt_sol=InterruptedRelax.RelaxDense(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,
    lmon_f,supp_f,coe_f,dg,dh,k,s,solver="Mosek",comp_opt_sol=false)

Relaxation order: k=1
Block size: s=50


Maximal matrix size:26
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 3276            
  Cones                  : 0               
  Scalar variables       : 2627            
  Matrix variables       : 26              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
GP based matrix reordering started.
GP based matrix reordering terminated.
Problem
  Name                   :             

(-91.28161291882756, Array{Float64,1}[])

In [107]:
using SparseArrays

include("../src/InterruptedRelax.jl")
using .InterruptedRelax

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=InterruptedRelax.get_info(x,f,g,h,sparse=true);

#InterruptedRelax.save_info_sparsePOP(randx,u,n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh)

#include("./sparsePOP3.jl")

using TSSOS

k=1

println("Maximal matrix size: ",binomial(n+k,k))

vars,pop=InterruptedRelax.get_POP(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f)

@time opt,sol,data=TSSOS.cs_tssos_first(pop,vars,k,numeq=l,CS=false,TS=false,solver="Mosek");

Maximal matrix size: 50
************************TSSOS************************
TSSOS is launching...
------------------------------------------------------
The clique sizes of varibles:
[49]
[1]
------------------------------------------------------
Assembling the SDP...
There are 1275 affine constraints.
SDP assembling time: 0.001674079 seconds.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 1275            
  Cones                  : 0               
  Scalar variables       : 51              
  Matrix variables       : 1               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time     

In [108]:
include("../src/InterruptedRelax.jl")
using .InterruptedRelax

InterruptedRelax.save_info_mat(n,A)

In [ ]:
# include("../src/InterruptedRelax.jl")
# using .InterruptedRelax

# n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=InterruptedRelax.get_info(x,f,g,h,sparse=false);


# k=2

# println("Relaxation order: k=",k)

# s=2

# println("Block size: s=",s)

# @time opt_val1=InterruptedRelax.RelaxDense_without_multiplier(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,
#     lmon_f,supp_f,coe_f,dg,dh,k,s,solver="Mosek")